<a href="https://colab.research.google.com/github/JuanT-17/ABP_Sprint2_CRUD/blob/main/Sprint2_CRUDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## **SISTEMA DE RECOMENDACIÓN Y PREDICCIÓN DE PELÍCULAS POR CONTENIDO Y VALORACIONES DE USUARIO**



---



Para el desarrollo del trabajo, se optó por hacer uso de un dataset extraído de internet (url: https://www.kaggle.com/shivamb/netflix-shows) llamado *netflix_titles.csv*, el cual contiene películas y series de televisión de la plataforma de streaming Netflix. 

Este archivo .csv lo incluimos dentro de la carpeta sample_data para poder trabajar con él.





## Carga del dataset y filtrado

---


El dataset *netflix_titles.csv*, que se había guardado dentro de la carpeta sample_data, se carga y se guarda dentro de la variable originalData.

In [ ]:
import pandas as pd

originalData = pd.read_csv('/content/netflix_titles.csv')
originalData

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
...,...,...,...,...,...,...,...,...,...,...,...,...
6229,80000063,TV Show,Red vs. Blue,NaN,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,NaN,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil..."
6230,70286564,TV Show,Maron,NaN,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,NaN,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews..."
6231,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,NaN,NaN,NaN,NaN,2016,NaN,60 min,Movies,Nursery rhymes and original music for children...
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,NaN,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaN,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."



Dado que el proyecto está enfocado a la recomendación únicamente de películas y el dataset contiene tanto películas como series de televisión, se debe realizar un filtrado sobre éste. Para ello, nos quedamos con aquellas filas que tengan como valor 'Movie' en la columna 'type'.

In [ ]:
filteredMovies = (originalData["type"] == "Movie")
dataMovies = originalData[filteredMovies]
dataMovies.reset_index()

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
3,6,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada","September 8, 2017",2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,7,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,"September 8, 2017",2017,TV-MA,60 min,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4260,5577,80085438,Movie,Frank and Cindy,G.J. Echternkamp,NaN,United States,"April 1, 2016",2007,TV-MA,70 min,Documentaries,Frank was a rising pop star when he married Ci...
4261,5578,80085439,Movie,Frank and Cindy,G.J. Echternkamp,"Rene Russo, Oliver Platt, Johnny Simmons, Jane...",United States,"April 1, 2016",2015,R,102 min,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...
4262,5579,80011846,Movie,Iverson,Zatella Beatty,Allen Iverson,United States,"April 1, 2016",2014,NR,88 min,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...
4263,5580,80064521,Movie,Jeremy Scott: The People's Designer,Vlad Yudin,Jeremy Scott,United States,"April 1, 2016",2015,PG-13,109 min,Documentaries,The journey of fashion designer Jeremy Scott f...



Por otra parte, se optó por trabajar únicamente con las columnas show_id(id de la película), title (título de la película), listed_in (géneros de la película) y description (descripción de la película). Para ello, se eliminaron todas las columnas que no fueran a ser utilizadas. Finalmente, se renombraron los índices de las filas para que se adaptasen al nuevo dataset *dataMovies*

In [ ]:
dataMovies = dataMovies.drop(columns=['type','director','cast','country','date_added','release_year','rating','duration'])
dataMovies = dataMovies.reset_index()
dataMovies = dataMovies.drop(columns=['index'])
dataMovies

,show_id,title,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
...,...,...,...,...
4260,80085438,Frank and Cindy,Documentaries,Frank was a rising pop star when he married Ci...
4261,80085439,Frank and Cindy,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...
4262,80011846,Iverson,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...
4263,80064521,Jeremy Scott: The People's Designer,Documentaries,The journey of fashion designer Jeremy Scott f...



El dataset ya filtrado se guarda dentro de la carpeta sample_data como dataMovies.csv

In [ ]:
dataMovies.to_csv('dataMovies.csv')
print(dataMovies)

       show_id  ...                                        description
0     81145628  ...  Before planning an awesome wedding for his gra...
1     80117401  ...  Jandino Asporaat riffs on the challenges of ra...
2     80125979  ...  When nerdy high schooler Dani finally attracts...
3     70304989  ...  In a dystopian future, an insurance adjuster f...
4     80164077  ...  Fabrizio Copano takes audience participation t...
...        ...  ...                                                ...
4260  80085438  ...  Frank was a rising pop star when he married Ci...
4261  80085439  ...  A student filmmaker vengefully turns his camer...
4262  80011846  ...  This unfiltered documentary follows the rocky ...
4263  80064521  ...  The journey of fashion designer Jeremy Scott f...
4264  80116008  ...  Nursery rhymes and original music for children...

[4265 rows x 4 columns]



## Procesado mediante la descripción de las películas


---



Para el procesado de las películas, cargamos el archivo .csv que habíamos guardado previamente y lo guardamos dentro de la variable originalData.

Todo este proceso se hace a través de la función LoadData().

In [ ]:
import pandas as pd

def LoadData():
  originalData = pd.read_csv('/content/dataMovies.csv')
  originalData = originalData.drop(columns=['Unnamed: 0'])
  return originalData

originalData = LoadData()
originalData

,show_id,title,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
...,...,...,...,...
4260,80085438,Frank and Cindy,Documentaries,Frank was a rising pop star when he married Ci...
4261,80085439,Frank and Cindy,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...
4262,80011846,Iverson,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...
4263,80064521,Jeremy Scott: The People's Designer,Documentaries,The journey of fashion designer Jeremy Scott f...



Dado que queremos filtrar en base a la descripción de la película, indicamos que la columna donde se encuentra la descripción es la 4. Como resultado del filtrado, se obtiene el dataset preprocessedData, que contiene una nueva columna llamada processed_text en la que se guardan las palabras más relevantes de la descripción para la creación de una bolsa de palabras. 

Todo este proceso está contenido dentro de la función ProcessedText().

In [ ]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

def ProcessedText():
  ps = PorterStemmer()

  preprocessedText = []

  for row in originalData.itertuples():
    text = word_tokenize(row[4]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    preprocessedText.append(text)

  preprocessedData = originalData
  preprocessedData['processed_text'] = preprocessedText
  return preprocessedData

preprocessedData = ProcessedText()
preprocessedData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,show_id,title,listed_in,description,processed_text
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...,befor plan awesom wed grandfath polar bear kin...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...,jandino asporaat riff challeng rais kid serena...
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...,when nerdi high schooler dani final attract in...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f...",In dystopian futur insur adjust tech compani i...
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...,fabrizio copano take audienc particip next lev...
...,...,...,...,...,...
4260,80085438,Frank and Cindy,Documentaries,Frank was a rising pop star when he married Ci...,frank rise pop star marri cindi decad dash dre...
4261,80085439,Frank and Cindy,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...,A student filmmak veng turn camera bitter mom ...
4262,80011846,Iverson,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...,thi unfilt documentari follow rocki life strat...
4263,80064521,Jeremy Scott: The People's Designer,Documentaries,The journey of fashion designer Jeremy Scott f...,the journey fashion design jeremi scott humbl ...



A partir de preprocessedData y su campo processed_text se crea una 'prematriz'.
Esta prematriz estará compuesta por textsBows que se crean haciendo uso de la librería. Estos textsBows almacenan la información de las frecuencias relativas de cada palabra.

Todo este proceso se guarda dentro de la función TextsBow().

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def TextsBoW():
  bagOfWordsModel = TfidfVectorizer()
  bagOfWordsModel.fit(preprocessedData['processed_text'])
  textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
  return textsBoW

textsBoW = TextsBoW()
print("Finished")

Finished


In [ ]:
print(textsBoW)

  (0, 9277)	0.2168627715614187
  (0, 8379)	0.14618748445900767
  (0, 8124)	0.25348965803276163
  (0, 6473)	0.2990451081940333
  (0, 6426)	0.19158260438077226
  (0, 5691)	0.1629066062898274
  (0, 4713)	0.2341420513582599
  (0, 3677)	0.2716509959831617
  (0, 3204)	0.196674253379278
  (0, 2936)	0.21446680467392523
  (0, 886)	0.3062507022227642
  (0, 862)	0.2788565900118926
  (0, 738)	0.18592706587739674
  (0, 723)	0.3264392204049048
  (0, 602)	0.3150696365779407
  (0, 545)	0.2990451081940333
  (1, 7685)	0.190003773679168
  (1, 7591)	0.21269938546178643
  (1, 7568)	0.3396415077898349
  (1, 7271)	0.31247315023733385
  (1, 7160)	0.255760213388685
  (1, 7032)	0.32374903741543737
  (1, 6822)	0.2184860750937684
  (1, 4697)	0.21347556240364013
  (1, 4486)	0.3396415077898349
  :	:
  (4263, 5532)	0.318460610004583
  (4263, 4591)	0.19987288981640763
  (4263, 4521)	0.33409347751923113
  (4263, 4120)	0.2590681425464476
  (4263, 3085)	0.25158238066402716
  (4263, 3077)	0.2650114829395983
  (4263, 2495


LLegados a este punto, únicamente queda por hacer el cálculo de distancias a través de un algoritmo. Se optó por hacer uso de la *Similitud coseno* para obtener las distancias entre **vectores**.

Todo este proceso se hace dentro de la función Distance().

In [ ]:
from sklearn.metrics import pairwise_distances

def Distance():
  distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')
  return distance_matrix

distance_matrix = Distance()


Con la prematriz con las frecuencias relativas crearemos una matriz NxN, donde N es el número de películas.
Cada celda devuelve la distancia entre las películas que representan cada índice, es decir i, j.

In [ ]:
print(distance_matrix.shape)
print(type(distance_matrix))

(4265, 4265)
<class 'numpy.ndarray'>


## Ejemplo de recomendación de películas


---


A continuación, se muestra un ejemplo de uso del sistema de Recomendación. Se seleccinó la película *Automata* para obtener una lista de recomendaciones de películas que guardan cierta similitud con ésta.

In [ ]:
searchTitle = "Automata" #Película base para las recomendaciones
indexOfTitle = preprocessedData[preprocessedData['title']==searchTitle].index.values[0]
indexOfTitle

3


Distancia entre la película Automata con el resto de la matriz.
1.0 -> Ningún parecido.
0.0 -> Completamente igual.

In [ ]:
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
distance_scores

[(0, 1.0),
 (1, 1.0),
 (2, 1.0),
 (3, 0.0),
 (4, 1.0),
 (5, 1.0),
 (6, 1.0),
 (7, 1.0),
 (8, 1.0),
 (9, 1.0),
 (10, 1.0),
 (11, 0.9778958181748679),
 (12, 1.0),
 (13, 1.0),
 (14, 1.0),
 (15, 1.0),
 (16, 1.0),
 (17, 1.0),
 (18, 1.0),
 (19, 0.9775934966601066),
 (20, 1.0),
 (21, 0.946499391935903),
 (22, 1.0),
 (23, 1.0),
 (24, 0.9262129792915725),
 (25, 1.0),
 (26, 0.9799492981842617),
 (27, 1.0),
 (28, 0.959111430216024),
 (29, 1.0),
 (30, 0.9755778523770584),
 (31, 1.0),
 (32, 1.0),
 (33, 1.0),
 (34, 1.0),
 (35, 1.0),
 (36, 1.0),
 (37, 1.0),
 (38, 1.0),
 (39, 1.0),
 (40, 1.0),
 (41, 1.0),
 (42, 0.934698977348553),
 (43, 1.0),
 (44, 1.0),
 (45, 1.0),
 (46, 1.0),
 (47, 1.0),
 (48, 1.0),
 (49, 1.0),
 (50, 1.0),
 (51, 1.0),
 (52, 1.0),
 (53, 1.0),
 (54, 1.0),
 (55, 1.0),
 (56, 1.0),
 (57, 1.0),
 (58, 1.0),
 (59, 1.0),
 (60, 1.0),
 (61, 1.0),
 (62, 1.0),
 (63, 1.0),
 (64, 1.0),
 (65, 1.0),
 (66, 1.0),
 (67, 0.9630563895808035),
 (68, 1.0),
 (69, 0.9383692226181054),
 (70, 1.0),
 (71, 1.0),



---


Ordenadas las distancias. La primera es la propia película Automata, por eso su diferencia es 0.0.

In [ ]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
ordered_scores

[(3, 0.0),
 (3128, 0.7780576751772714),
 (2176, 0.8038845560227446),
 (886, 0.8448292557446013),
 (2566, 0.8482385810577173),
 (1858, 0.849020637599637),
 (3492, 0.8656052390137535),
 (3668, 0.8760996072912017),
 (3471, 0.8809011018460631),
 (2297, 0.8858285485505504),
 (3423, 0.8871172577253242),
 (1200, 0.8873267834371862),
 (3483, 0.8927106150182245),
 (3270, 0.8943209296954553),
 (790, 0.8952947505791),
 (4101, 0.8961928320658743),
 (373, 0.8964277158689071),
 (2972, 0.8977596644777873),
 (939, 0.9004001207676523),
 (2814, 0.9009689153371075),
 (3162, 0.9011160080571714),
 (2949, 0.9035652571281456),
 (1237, 0.9037823710417417),
 (2784, 0.9061900411353669),
 (1307, 0.9064693797399047),
 (2425, 0.9069527146823415),
 (294, 0.908404942476529),
 (1759, 0.9094838660682354),
 (3509, 0.9107135925235761),
 (2100, 0.9110572261580074),
 (3098, 0.9116643563241169),
 (2070, 0.913339880623795),
 (3624, 0.9134012942508509),
 (3087, 0.9138721607892099),
 (1344, 0.9149111617030032),
 (3257, 0.9150


A continuación, se muestran las 10 películas más parecidas a Automata.

In [ ]:
top_scores = ordered_scores[1:11]
top_scores

[(3128, 0.7780576751772714),
 (2176, 0.8038845560227446),
 (886, 0.8448292557446013),
 (2566, 0.8482385810577173),
 (1858, 0.849020637599637),
 (3492, 0.8656052390137535),
 (3668, 0.8760996072912017),
 (3471, 0.8809011018460631),
 (2297, 0.8858285485505504),
 (3423, 0.8871172577253242)]

In [ ]:
top_indexes = [i[0] for i in top_scores]
top_indexes

[3128, 2176, 886, 2566, 1858, 3492, 3668, 3471, 2297, 3423]


Títulos de las 10 películas más parecidas.

In [ ]:
preprocessedData['title'].iloc[top_indexes]

3128                            Master
2176                        Ex Machina
886                      The Rainmaker
2566    Roger Corman's Death Race 2050
1858                       Equilibrium
3492                            Mantra
3668                            Icarus
3471                        F.R.E.D.I.
2297             Cities of Last Things
3423               The Humanity Bureau
Name: title, dtype: object

## Gestión CRUD


---



**AÑADIR PELÍCULA**

El añadido de una película se divide en tres partes. En la primera, se inserta la nueva película en el dataset de películas. En la segunda, se vuelve a realizar el cálculo de distancias para que esta nueva película sea tenida en cuenta a la hora de recomendar. En la última, se almacena la nueva tabla generada con la película insertada.

Ha de tenerse en cuenta que para que todo el código que viene a continuación funcione, deben haberse ejecutado todos los bloques de código de la sacción **"Procesado mediante la descripción de las películas"**. Esto es debido a que en dicha sección se definen varias funciones que serán usadas en ésta.



**1.Agregación de la Película al dataset**

Primero de todo, debe cargarse el dataset que contiene todas las películas.



In [ ]:
import pandas as pd

originalData = LoadData()
originalData

,show_id,title,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
...,...,...,...,...
4260,80085438,Frank and Cindy,Documentaries,Frank was a rising pop star when he married Ci...
4261,80085439,Frank and Cindy,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...
4262,80011846,Iverson,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...
4263,80064521,Jeremy Scott: The People's Designer,Documentaries,The journey of fashion designer Jeremy Scott f...



Una vez cargado el dataset, se añade la nueva fila y se comprueba que se haya agregado correctamente al dataset.

In [ ]:
codigo = hash('Alberto') % 100000000
existe = False

while existe == False:
  if (originalData['show_id'] == codigo).any():
    codigo = (codigo + 1) % 100000000
  else:
    existe = True

originalData = originalData.append({'show_id' : codigo , 'title' : 'Alberto', 'listed_in' : 'Comedy', 'description' : 'This film is for testing'} , ignore_index=True)

filtered = (originalData["show_id"] == codigo)
comprobacion = originalData[filtered]
comprobacion

,show_id,title,listed_in,description,processed_text
4268,20267480,Alberto,Comedy,This film is for testing,NaN


**2.Cálculo de distancias**


Ahora se debe actualizar la bolsa de palabras y realizar nuevamente el cálculo de distancias para que se tenga en cuenta la nueva película insertada.


In [ ]:
preprocessedData = ProcessedText()
textsBoW = TextsBoW()
distance_matrix = Distance()

print(distance_matrix.shape)
print(type(distance_matrix))

(4265, 4265)
<class 'numpy.ndarray'>



A continuación, se incluye un ejemplo de cómo el sistema es capaz de recomendar películas semejantes a la nueva película agragada.

In [ ]:
searchTitle = "Alberto" #Película base para las recomendaciones
indexOfTitle = preprocessedData[preprocessedData['title']==searchTitle].index.values[0]
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
top_scores = ordered_scores[1:11]
print(top_scores)
top_indexes = [i[0] for i in top_scores]
print(preprocessedData['title'].iloc[top_indexes])

[(4266, 0.0), (4267, 0.0), (4268, 0.0), (1533, 0.7698234845631124), (1212, 0.7914657638972493), (3314, 0.7943758023653108), (3554, 0.8010371215986466), (1971, 0.8037600759465194), (1870, 0.8064806397638378), (3726, 0.8130263476473186)]
4266                                           Prueba
4267                                           Prueba
4268                                          Alberto
1533    Undercover: How to Operate Behind Enemy Lines
1212                                  Krishna Cottage
3314                                After Porn Ends 3
3554                  Martin Lawrence Live: Runteldat
1971                                               XX
1870                              The Black Godfather
3726                                     The Congress
Name: title, dtype: object



**3.Almacenamiento del Dataset**


Finalmente, se guarda el dataset con la nueva película agregada.

In [ ]:
originalData.to_csv('originalData.csv')
print(originalData)

       show_id  ...                                     processed_text
0     81145628  ...  befor plan awesom wed grandfath polar bear kin...
1     80117401  ...  jandino asporaat riff challeng rais kid serena...
2     80125979  ...  when nerdi high schooler dani final attract in...
3     70304989  ...  In dystopian futur insur adjust tech compani i...
4     80164077  ...  fabrizio copano take audienc particip next lev...
...        ...  ...                                                ...
4260  80085438  ...  frank rise pop star marri cindi decad dash dre...
4261  80085439  ...  A student filmmak veng turn camera bitter mom ...
4262  80011846  ...  thi unfilt documentari follow rocki life strat...
4263  80064521  ...  the journey fashion design jeremi scott humbl ...
4264  80116008  ...  nurseri rhyme origin music children accompani ...

[4265 rows x 5 columns]




---


**MODIFICAR PELÍCULA**

Al igual que pasaba en *Añadir Película*, la modificación de una película también se divide en tres partes: modificación de la película; actualización de las distancias; y guardado del dataset modificado.


Ha de tenerse en cuenta que para que todo el código que viene a continuación funcione, deben haberse ejecutado todos los bloques de código de la sacción **"Procesado mediante la descripción de las películas"**. Esto es debido a que en dicha sección se definen varias funciones que serán usadas en ésta.

**1.Modificación de la Película en el dataset**

Primero de todo, debe cargarse el dataset que contiene todas las películas.

In [ ]:
import pandas as pd

originalData = LoadData()
originalData

,show_id,title,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
...,...,...,...,...
4260,80085438,Frank and Cindy,Documentaries,Frank was a rising pop star when he married Ci...
4261,80085439,Frank and Cindy,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...
4262,80011846,Iverson,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...
4263,80064521,Jeremy Scott: The People's Designer,Documentaries,The journey of fashion designer Jeremy Scott f...



Se busca la película a editar y se modifican sus datos.

In [ ]:
codigo = 80117401
existe = False

originalData.loc[originalData['show_id'] == codigo, ['description']] = 'The description\'s value changed '
originalData

,show_id,title,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Stand-Up Comedy,The description's value changed
2,80125979,#realityhigh,Comedies,When nerdy high schooler Dani finally attracts...
3,70304989,Automata,"International Movies, Sci-Fi & Fantasy, Thrillers","In a dystopian future, an insurance adjuster f..."
4,80164077,Fabrizio Copano: Solo pienso en mi,Stand-Up Comedy,Fabrizio Copano takes audience participation t...
...,...,...,...,...
4260,80085438,Frank and Cindy,Documentaries,Frank was a rising pop star when he married Ci...
4261,80085439,Frank and Cindy,"Comedies, Dramas, Independent Movies",A student filmmaker vengefully turns his camer...
4262,80011846,Iverson,"Documentaries, Sports Movies",This unfiltered documentary follows the rocky ...
4263,80064521,Jeremy Scott: The People's Designer,Documentaries,The journey of fashion designer Jeremy Scott f...


**2.Cálculo de distancias**

Ahora se debe actualizar la bolsa de palabras para que se tenga en cuenta la modificación realizada.

In [ ]:
preprocessedData = ProcessedText()
textsBoW = TextsBoW()
distance_matrix = Distance()

print(distance_matrix.shape)
print(type(distance_matrix))

**3.Almacenamiento del Dataset**

Finalmente, se guarda en el dataset las modificaciones agregadas a la película.

In [ ]:
originalData.to_csv('originalData.csv')
print(originalData)